In [1]:
"""將PTT目標版面的文章、關鍵字標題、希望爬取頁數,將網址download至'pttFile.csv'檔案中"""

def pttURL_crawler(board,keyWd,pageNo,writeFile):
    import requests
    from bs4 import BeautifulSoup
    import csv

    """擷取ptt文章列表的網頁,擷取每篇文章的連結
       並翻每一頁網頁,爬取想要的頁數"""

    URL = "https://www.ptt.cc/"
    headers = {"cookies" : "over18=1"}
    #希望爬取ptt板面、但是頁面擷取的網址內容會缺乏URL,需要自行補上
    res = requests.get(URL + "bbs/" + board +"/index.html" , headers = headers)
    soup = BeautifulSoup(res.text , 'lxml')

    bottons = soup.select('a.btn.wide')
    #totalpage=上一頁的頁數+1
    totalpage = int(bottons[1]['href'].split('index')[1].split('.')[0])+1

    """在希望的頁數（totalpage ~ crawlpage-n)的文章列表中,希望爬出文章標題為"KeyWord"的網址
       所以,只要爬到該文章就將網址寫入'pttFile.csv'
    """
    
    for page in range(totalpage,totalpage - pageNo, -1):
    
        res = requests.get(URL+ "bbs/" + board +"/index{}.html".format(page),headers = headers)    
        soup = BeautifulSoup(res.text,'lxml')
        links = soup.select('div.title > a')
        
        #爬取該頁面中每一篇有'keyword'的文章
        for link in links:
            title = link.text
            if(keyWd) in title:        
                article_URL = URL +link['href']
                print(article_URL) 
                #只要爬到該keyword的文章寫入'pttFile.csv'
                with open(writeFile,'a') as fw:
                    fw.write(article_URL + '\n')
    
    return print("download finished!")

In [1]:
"""將csv檔案中的網址（ptt）內容爬出（限定只有英文的部份）並寫入檔案中"""

def pttEng_crawler(readFile,writeFile):
    import requests
    from bs4 import BeautifulSoup
    import csv
    import re

    """#將檔案中的網址讀取,並進入該網址（ptt）的頁面"""
    with open(readFile,'r') as fr:
        for URL in fr:
            article_URL = URL.strip()        
            res   = requests.get(article_URL,headers = {"cookies" : "over18=1"})
            soup  = BeautifulSoup(res.text,"lxml") 
            links = soup.select("div.article-metaline > span")
            
            #預期將所有文章的英文串接
            """將每篇文章的內容爬出,並過濾爬出只有英文字的內容,每篇文章的英文字會以'*'串接,並寫入檔案中"""
            for link in links:

                words = []    
                article_dict ={}   
                #去除文章本文以外的標籤    
                trash_link = soup.select('div.article-metaline > span') + soup.select('div.article-metaline-right > span') + soup.select('div.push') + soup.select('div#article-polling') 
                for trash in trash_link:
                    trash.decompose()
                #過濾文章內文,並將有英文的部份全部改為小寫    
                content = soup.select_one("#main-container")
                for script in content.find_all('script'):
                    script.decompose()
                for a in content.select('a'):
                    a.decompose()
                article_dict['content'] =content.text
                content = article_dict['content'].lower()
                #過濾文章內容的特殊符號

                clearContent = content.replace('\n', '').replace('(', '').replace(')', '').replace(',', '').replace('.','')
                #限定文章中只有英文是我們想要的內容
                word = list(set(re.findall('object c|visual basic|[A-Za-z.+#]+',clearContent, re.IGNORECASE)))
                #該篇文章所有的英文字串接為words
    #                 print(word)
                words += word

            with open(writeFile,'a') as fw:
                fw.write('*'.join(words) + '\n')
                
    return print("get the English part of these article!")

In [2]:
"""將想要比較（本文文章）和被比較的檔案（有keyWord的斷詞）,相互比較得到keyWord出現的次數"""

def CountKeyWd(compareFile,comparedFile,result):
    from collections import Counter
    import re
    #open 被比較的檔案（含有keyWord的斷詞）
    with open(comparedFile,'r') as f:
        lan = [text.lower() for text in f.read().split('\n')]
    #open 想要比較的本文檔案  
    with open(compareFile,'r') as fr:        
        wc = Counter()
        """只要本文檔案,每讀取一行（words)就經過處理成為好幾個word,
        並判斷該word是否有在被比較檔案中（含有keyWord斷詞）,
        如果有就value+1,反之就value為1"""
        for strWd in fr:
            words  = strWd.strip().split('*')       
            
            for word in words:
                if word in lan:
                    if word in wc:
                        wc[word] += 1
                    else: 
                        wc[word] = 1        
    """計算出各關鍵字出現數量就將檔案寫入'result'csv檔案中"""
    with open(result, "w") as fw:
        for k,v in  wc.most_common():
            fw.write('{},{}'.format(k,v)  + '\n')
    return print(wc.most_common())

In [3]:
pttEng_crawler("softURL.csv","softEng.csv")
CountKeyWd('softEng.csv','lan.csv','newSoftCount.csv')

get the English part of these article!
[('javascript', 713), ('html', 677), ('java', 670), ('php', 650), ('android', 536), ('c', 530), ('linux', 516), ('mysql', 470), ('jquery', 466), ('ios', 439), ('c++', 404), ('c#', 282), ('python', 278), ('ajax', 272), ('ruby', 164), ('project', 140), ('oracle', 121), ('unix', 115), ('xml', 102), ('flash', 102), ('jsp', 93), ('http', 91), ('spring', 81), ('r', 75), ('asp', 69), ('perl', 67), ('shell', 63), ('illustrator', 62), ('excel', 57), ('postgresql', 55), ('oop', 54), ('socket', 49), ('swift', 46), ('apple', 45), ('security', 43), ('arm', 43), ('struts', 43), ('actionscript', 42), ('uml', 37), ('base', 37), ('matlab', 36), ('access', 33), ('word', 32), ('dhtml', 30), ('udp', 29), ('flex', 28), ('ideas', 27), ('tomcat', 27), ('dreamweaver', 27), ('firmware', 25), ('rdbms', 25), ('xhtml', 24), ('dns', 23), ('iis', 21), ('mfc', 20), ('firewall', 19), ('voip', 18), ('san', 18), ('assembly', 18), ('vmware', 17), ('dsp', 16), ('jsf', 16), ('soap', 

In [5]:
pttEng_crawler("techURL.csv","techEng.csv")
CountKeyWd('techEng.csv','lan.csv','newTechCount.csv')

get the English part of these article!
[('c', 235), ('c++', 193), ('linux', 188), ('android', 179), ('java', 178), ('ios', 125), ('html', 121), ('javascript', 118), ('php', 115), ('c#', 90), ('python', 89), ('project', 88), ('mysql', 82), ('jquery', 72), ('ruby', 60), ('r', 49), ('matlab', 41), ('unix', 40), ('word', 38), ('excel', 37), ('ajax', 33), ('security', 30), ('oracle', 26), ('perl', 25), ('http', 24), ('fpga', 24), ('base', 22), ('shell', 22), ('vmware', 21), ('rf', 20), ('illustrator', 19), ('verilog', 18), ('drivers', 18), ('xml', 18), ('flash', 17), ('arm', 15), ('powerpoint', 15), ('spring', 15), ('dsp', 15), ('access', 14), ('dns', 14), ('labview', 14), ('mcu', 13), ('eda', 12), ('firmware', 11), ('autocad', 11), ('wan', 11), ('swift', 10), ('iis', 10), ('udp', 10), ('ideas', 10), ('bluetooth', 9), ('solidworks', 9), ('sap', 9), ('apple', 8), ('xhtml', 8), ('uml', 8), ('plc', 8), ('vpn', 8), ('firewall', 7), ('san', 7), ('rdbms', 7), ('mes', 7), ('socket', 7), ('director